In [4]:
!ls /kaggle/input

faces-in-wild


In [3]:
!pwd

/kaggle/working


In [2]:
!tar -xvf /kaggle/input/faces-in-wild/lfw-deepfunneled.tgz -C .

lfw-deepfunneled/AJ_Cook/AJ_Cook_0001.jpg
lfw-deepfunneled/AJ_Lamas/AJ_Lamas_0001.jpg
lfw-deepfunneled/Aaron_Eckhart/Aaron_Eckhart_0001.jpg
lfw-deepfunneled/Aaron_Guiel/Aaron_Guiel_0001.jpg
lfw-deepfunneled/Aaron_Patterson/Aaron_Patterson_0001.jpg
lfw-deepfunneled/Aaron_Peirsol/Aaron_Peirsol_0001.jpg
lfw-deepfunneled/Aaron_Peirsol/Aaron_Peirsol_0002.jpg
lfw-deepfunneled/Aaron_Peirsol/Aaron_Peirsol_0003.jpg
lfw-deepfunneled/Aaron_Peirsol/Aaron_Peirsol_0004.jpg
lfw-deepfunneled/Aaron_Pena/Aaron_Pena_0001.jpg
lfw-deepfunneled/Aaron_Sorkin/Aaron_Sorkin_0001.jpg
lfw-deepfunneled/Aaron_Sorkin/Aaron_Sorkin_0002.jpg
lfw-deepfunneled/Aaron_Tippin/Aaron_Tippin_0001.jpg
lfw-deepfunneled/Abba_Eban/Abba_Eban_0001.jpg
lfw-deepfunneled/Abbas_Kiarostami/Abbas_Kiarostami_0001.jpg
lfw-deepfunneled/Abdel_Aziz_Al-Hakim/Abdel_Aziz_Al-Hakim_0001.jpg
lfw-deepfunneled/Abdel_Madi_Shabneh/Abdel_Madi_Shabneh_0001.jpg
lfw-deepfunneled/Abdel_Nasser_Assidi/Abdel_Nasser_Assidi_0001.jpg
lfw-deepfunneled/Abdel_Nasser_

In [3]:
!ls /kaggle/working

lfw-deepfunneled


In [5]:
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, IterableDataset
from PIL import Image
from typing import Optional


class Preprocess:
	pairs_dev_train_path = '../../pairsDevTrain.txt'
	pairs_dev_test_path = '../../pairsDevTest.txt'
	people_dev_train_path = '../../peopleDevTrain.txt'
	people_dev_test_path = '../../peopleDevTest.txt'
	dataset_path = '../../lfw'

	@staticmethod
	def get_image_path(name, num, img_dir = dataset_path):
		return f'{img_dir}/{name}/{name}_{num:0>4}.jpg'
	
	@staticmethod
	def get_image(name, num, img_dir = dataset_path):
		return Image.open(Preprocess.get_image_path(name, num, img_dir)).convert("RGB")

	@staticmethod
	def extract_pairs(filepath: str):
		pairs = []
		with open(filepath, 'r') as f:
			size = int(next(f))
			for i in range(size):
				line = next(f)
				name, num1, num2 = line.strip().split()
				pairs.append(((name, int(num1)), (name, int(num2))))
			for i in range(size):
				line = next(f)
				name1, num1, name2, num2 = line.strip().split()
				pairs.append(((name1, int(num1)), (name2, int(num2))))
		return pairs
	
	@staticmethod
	def extract_people(filepath: str):
		people = {}
		with open(filepath, 'r') as f:
			size = int(next(f))
			for i in range(size):
				line = next(f)
				name, num = line.strip().split()
				people[name] = int(num)
		return people
	
	@staticmethod
	def load_train_pairs(transform=None):
		train_pairs = Preprocess.extract_pairs(Preprocess.pairs_dev_train_path)
		return PairDataGenerator(train_pairs, transform=transform)

	@staticmethod
	def load_test_pairs(transform=None):
		test_pairs = Preprocess.extract_pairs(Preprocess.pairs_dev_test_path)
		return PairDataGenerator(test_pairs, transform=transform)
	
	@staticmethod
	def load_sample_pairs(size: int = 20, transform=None):
		train_pairs = Preprocess.extract_pairs(Preprocess.pairs_dev_train_path)
		return PairDataGenerator(train_pairs[:size], transform=transform)
	
	@staticmethod
	def load_train_people(shuffle=False, seed: Optional[int] = None, transform=None):
		train_people = Preprocess.extract_people(Preprocess.people_dev_train_path)
		return ImageGenerator(train_people, transform=transform, shuffle=shuffle, seed=seed)

	@staticmethod
	def load_test_people():
		test_people = Preprocess.extract_people(Preprocess.people_dev_test_path)
		return ImageGenerator(test_people)
		
		
class ImageGenerator(IterableDataset):
	def __init__(self, 
			people, 
			img_dir = Preprocess.dataset_path, 
			transform=None, 
			output_name=False, 
			shuffle=False,
			seed: Optional[int] =None
		):
		self.people = people
		self.people_order = list(people.keys())
		self.img_dir = img_dir
		self.transform = transforms.ToTensor() if transform is None else transform
		self.output_name = output_name
		self.shuffle = shuffle
		self.rng = np.random.default_rng(seed=seed)
	
	def __len__(self):
		return np.sum(list(self.people.values()))
	
	def __iter__(self):
		if self.shuffle:
			self.rng.shuffle(self.people_order)
		for name in self.people_order:
			image_count = self.people[name]
			image_order = np.arange(1, image_count + 1)
			if self.shuffle:
				self.rng.shuffle(image_order)
			for num in image_order:
				image = Preprocess.get_image(name, num, img_dir=self.img_dir)
				image = self.transform(image)
				if self.output_name:
					yield image, name
				else:
					yield image

class PairDataGenerator(Dataset):
	def __init__(self, pairs, img_dir = Preprocess.dataset_path, transform=None):
		self.pairs = pairs
		self.img_dir = img_dir
		self.transform = transforms.ToTensor() if transform is None else transform

	def __len__(self):
		return len(self.pairs)

	def __getitem__(self, index):
		(name1, num1), (name2, num2) = self.pairs[index]
		# Load images
		image1 = Preprocess.get_image(name1, num1, img_dir=self.img_dir)
		image2 = Preprocess.get_image(name2, num2, img_dir=self.img_dir)
		# Transform images
		image1 = self.transform(image1)
		image2 = self.transform(image2)
		# Create label
		y_true = 1 if name1 == name2 else 0
		return image1, image2, y_true

In [5]:
!pip install timm

In [2]:
import timm
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
import functools
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin: float = 1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x1, x2, label):
        dist = nn.functional.pairwise_distance(x1, x2)
        # label 1 means similar, 0 means dissimilar
        # when similar, loss is the distance
        # when dissimilar and more distant than the margin, no loss
        # when dissimilar and closer than the margin, loss is the distance to the margin
        loss = label * torch.pow(dist, 2) + (1 - label) * torch.pow(torch.clamp(self.margin - dist, min=0.0), 2)
        loss = torch.mean(loss)
        return loss

In [6]:
class TimmSiameseNetwork(nn.Module):
    def __init__(self):
        super(TimmSiameseNetwork, self).__init__()
        # https://huggingface.co/timm/vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k
        self.model = timm.create_model(
            'vit_mediumd_patch16_reg4_gap_256.sbb_in12k_ft_in1k',
            pretrained=True,
            num_classes=0,  # remove classifier nn.Linear
        )
        # Freeze all layers except the last conv
        for param in self.model.parameters():
            param.requires_grad = False
        for param in self.model.blocks[-1].parameters():
            param.requires_grad = True

        # get model specific transforms (normalization, resize)
        data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(**data_config, is_training=True)

    def forward_once(self, img) -> torch.Tensor:
        return self.model(self.transforms(img))

    def forward(self, img1, img2):
        return self.forward_once(img1), self.forward_once(img2)

timm_model = TimmSiameseNetwork()

In [11]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 20

train_dataset = Preprocess.load_train_pairs()
val_dataset = Preprocess.load_test_pairs()

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = ContrastiveLoss()
optimizer = optim.Adam(timm_model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Training loop
def train_loop(model):
    best_val_loss = float('inf')
    model = model.to(device)

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0

        for batch_idx, (img1, img2, labels) in enumerate(train_loader):
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

            optimizer.zero_grad()
            output1, output2 = model(img1, img2)
            loss = criterion(output1, output2, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            print(f'Epoch {epoch + 1}, Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}     ', end='\r')

        avg_train_loss = train_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for img1, img2, labels in val_loader:
                img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
                output1, output2 = model(img1, img2)
                loss = criterion(output1, output2, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        # Print epoch results
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Learning rate scheduling
        scheduler.step(avg_val_loss)

        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_siamese_model.pth')

    print("Training completed!")

train_loop(timm_model)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/20], Train Loss: 68.7688, Val Loss: 1.6701
Epoch [2/20], Train Loss: 1.2127, Val Loss: 0.8733
Epoch [3/20], Train Loss: 0.6687, Val Loss: 0.5027
Epoch [4/20], Train Loss: 0.4425, Val Loss: 0.3672
Epoch [5/20], Train Loss: 0.3389, Val Loss: 0.2909
Epoch [6/20], Train Loss: 0.2750, Val Loss: 0.2542
Epoch [7/20], Train Loss: 0.2385, Val Loss: 0.2275
Epoch [8/20], Train Loss: 0.2232, Val Loss: 0.2126
Epoch [9/20], Train Loss: 0.2164, Val Loss: 0.2165
Epoch [10/20], Train Loss: 0.2094, Val Loss: 0.2075
Epoch [11/20], Train Loss: 0.2033, Val Loss: 0.2057
Epoch [12/20], Train Loss: 0.2017, Val Loss: 0.1951
Epoch [13/20], Train Loss: 0.1947, Val Loss: 0.1857
Epoch [14/20], Train Loss: 0.1908, Val Loss: 0.1870
Epoch [15/20], Train Loss: 0.1855, Val Loss: 0.1871
Epoch [16/20], Train Loss: 0.1835, Val Loss: 0.1753
Epoch [17/20], Train Loss: 0.1833, Val Loss: 0.1763
Epoch [18/20], Train Loss: 0.1769, Val Loss: 0.1704
Epoch [19/20], Train Loss: 0.1718, Val Loss: 0.1702
Epoch [20/20], Train

In [13]:
train_loop(timm_model)

Epoch [1/20], Train Loss: 0.1668, Val Loss: 0.1597
Epoch [2/20], Train Loss: 0.1654, Val Loss: 0.1545
Epoch [3/20], Train Loss: 0.1586, Val Loss: 0.1605
Epoch [4/20], Train Loss: 0.1566, Val Loss: 0.1600
Epoch [5/20], Train Loss: 0.1503, Val Loss: 0.1468
Epoch [6/20], Train Loss: 0.1529, Val Loss: 0.1564
Epoch [7/20], Train Loss: 0.1461, Val Loss: 0.1504
Epoch [8/20], Train Loss: 0.1466, Val Loss: 0.1440
Epoch [9/20], Train Loss: 0.1459, Val Loss: 0.1368
Epoch [10/20], Train Loss: 0.1386, Val Loss: 0.1426
Epoch [11/20], Train Loss: 0.1415, Val Loss: 0.1417
Epoch [12/20], Train Loss: 0.1422, Val Loss: 0.1432
Epoch [13/20], Train Loss: 0.1389, Val Loss: 0.1393
Epoch [14/20], Train Loss: 0.1467, Val Loss: 0.1382
Epoch [15/20], Train Loss: 0.1395, Val Loss: 0.1480
Epoch [16/20], Train Loss: 0.1364, Val Loss: 0.1380
Epoch [17/20], Train Loss: 0.1345, Val Loss: 0.1280
Epoch [18/20], Train Loss: 0.1306, Val Loss: 0.1416
Epoch [19/20], Train Loss: 0.1353, Val Loss: 0.1381
Epoch [20/20], Train 

In [15]:
train_loop(timm_model)

Epoch [1/20], Train Loss: 0.1313, Val Loss: 0.1358
Epoch [2/20], Train Loss: 0.1305, Val Loss: 0.1434
Epoch [3/20], Train Loss: 0.1275, Val Loss: 0.1451
Epoch [4/20], Train Loss: 0.1271, Val Loss: 0.1386
Epoch [5/20], Train Loss: 0.1312, Val Loss: 0.1385
Epoch [6/20], Train Loss: 0.1261, Val Loss: 0.1404
Epoch [7/20], Train Loss: 0.1285, Val Loss: 0.1412
Epoch [8/20], Train Loss: 0.1331, Val Loss: 0.1342
Epoch [9/20], Train Loss: 0.1306, Val Loss: 0.1291
Epoch [10/20], Train Loss: 0.1257, Val Loss: 0.1412
Epoch [11/20], Train Loss: 0.1265, Val Loss: 0.1489
Epoch [12/20], Train Loss: 0.1328, Val Loss: 0.1520
Epoch [13/20], Train Loss: 0.1292, Val Loss: 0.1274
Epoch [14/20], Train Loss: 0.1298, Val Loss: 0.1398
Epoch [15/20], Train Loss: 0.1293, Val Loss: 0.1328
Epoch [16/20], Train Loss: 0.1307, Val Loss: 0.1435
Epoch [17/20], Train Loss: 0.1279, Val Loss: 0.1302
Epoch [18/20], Train Loss: 0.1311, Val Loss: 0.1402
Epoch [19/20], Train Loss: 0.1266, Val Loss: 0.1392
Epoch [20/20], Train 

In [16]:
!mv best_siamese_model.pth best_siamese_model_53epochs

In [7]:
train_data = Preprocess.load_train_pairs()
val_data = Preprocess.load_test_pairs()

In [11]:
model = TimmSiameseNetwork()
model.load_state_dict(torch.load('../../trained/timm-siamese/best_siamese_model-37epochs.pth', map_location=torch.device('cpu')))
model

C:\Users\Renan\AppData\Local\Temp\ipykernel_736\986382908.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../../trained/timm-siamese/be

TimmSiameseNetwork(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 512, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (act): GELU(approximate

In [19]:
def predictions(data, model, threshold=0.5):
    model.eval()
    with torch.no_grad():
        pred = []
        y_true = []
        for i, (img1, img2, label) in enumerate(data):
            output1, output2 = model(img1.unsqueeze(0), img2.unsqueeze(0))
            euclidean_distance = nn.PairwiseDistance()(output1.squeeze(), output2.squeeze())
            prediction = int(euclidean_distance < threshold)
            pred.append(prediction)
            y_true.append(label)
            print(f'Progress: {i}/{len(data)}        ', end='\r')
        return pred, y_true


In [14]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

y_true, pred = predictions(val_data, model, 0.5)
accuracy = accuracy_score(y_true, pred)
recall = recall_score(y_true, pred)
precision = precision_score(y_true, pred)
f1 = f1_score(y_true, pred)
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1: {f1}')
print(f'ROC AUC: {roc_auc_score(y_true, pred)}')

Accuracy: 0.814
Recall: 0.848
Precision: 0.7940074906367042
F1: 0.8201160541586073
ROC AUC: 0.8140000000000001


In [15]:
pred_6, y_true_6 = predictions(val_data, model, 0.6)
print(f'Accuracy: {accuracy_score(y_true_6, pred_6)}')
print(f'Recall: {recall_score(y_true_6, pred_6)}')
print(f'Precision: {precision_score(y_true_6, pred_6)}')
print(f'F1: {f1_score(y_true_6, pred_6)}')
print(f'ROC AUC: {roc_auc_score(y_true_6, pred_6)}')

Accuracy: 0.819000        
Recall: 0.932
Precision: 0.7601957585644372
F1: 0.8373764600179695
ROC AUC: 0.819


In [20]:
model = TimmSiameseNetwork()
model.load_state_dict(torch.load('../../trained/timm-siamese/best_siamese_model-40epochs.pth', map_location=torch.device('cpu')))
model

C:\Users\Renan\AppData\Local\Temp\ipykernel_736\2828698963.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../../trained/timm-siamese/b

TimmSiameseNetwork(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 512, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (act): GELU(approximate

In [21]:
pred_40eps_5, y_true_40eps_5 = predictions(val_data, model, 0.5)
print(f'Accuracy: {accuracy_score(y_true_40eps_5, pred_40eps_5)}')
print(f'Recall: {recall_score(y_true_40eps_5, pred_40eps_5)}')
print(f'Precision: {precision_score(y_true_40eps_5, pred_40eps_5)}')
print(f'F1: {f1_score(y_true_40eps_5, pred_40eps_5)}')
print(f'ROC AUC: {roc_auc_score(y_true_40eps_5, pred_40eps_5)}')

Accuracy: 0.858000        
Recall: 0.88
Precision: 0.842911877394636
F1: 0.8610567514677103
ROC AUC: 0.8579999999999999


In [22]:
pred_40eps_6, y_true_40eps_6 = predictions(val_data, model, 0.6)
print(f'Accuracy: {accuracy_score(y_true_40eps_6, pred_40eps_6)}')
print(f'Recall: {recall_score(y_true_40eps_6, pred_40eps_6)}')
print(f'Precision: {precision_score(y_true_40eps_6, pred_40eps_6)}')
print(f'F1: {f1_score(y_true_40eps_6, pred_40eps_6)}')
print(f'ROC AUC: {roc_auc_score(y_true_40eps_6, pred_40eps_6)}')

Accuracy: 0.841000        
Recall: 0.934
Precision: 0.7861952861952862
F1: 0.8537477148080439
ROC AUC: 0.8400000000000001


In [23]:
model = TimmSiameseNetwork()
model.load_state_dict(torch.load('../../trained/timm-siamese/best_siamese_model-33epochs.pth', map_location=torch.device('cpu')))
model

C:\Users\Renan\AppData\Local\Temp\ipykernel_736\2697569088.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('../../trained/timm-siamese/b

TimmSiameseNetwork(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 512, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=512, out_features=1536, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=512, out_features=512, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (act): GELU(approximate

In [31]:
pred_33eps_5, y_true_33eps_5 = predictions(val_data, model, 0.5)
print(f'Accuracy: {accuracy_score(y_true_33eps_5, pred_33eps_5)}')
print(f'Recall: {recall_score(y_true_33eps_5, pred_33eps_5)}')
print(f'Precision: {precision_score(y_true_33eps_5, pred_33eps_5)}')
print(f'F1: {f1_score(y_true_33eps_5, pred_33eps_5)}')
print(f'ROC AUC: {roc_auc_score(y_true_33eps_5, pred_33eps_5)}')

Accuracy: 0.849000        
Recall: 0.868
Precision: 0.8362235067437379
F1: 0.8518155053974484
ROC AUC: 0.8489999999999999


In [32]:
pred_33eps_6, y_true_33eps_6 = predictions(val_data, model, 0.6)
print(f'Accuracy: {accuracy_score(y_true_33eps_6, pred_33eps_6)}')
print(f'Recall: {recall_score(y_true_33eps_6, pred_33eps_6)}')
print(f'Precision: {precision_score(y_true_33eps_6, pred_33eps_6)}')
print(f'F1: {f1_score(y_true_33eps_6, pred_33eps_6)}')
print(f'ROC AUC: {roc_auc_score(y_true_33eps_6, pred_33eps_6)}')

Accuracy: 0.846000        
Recall: 0.936
Precision: 0.7932203389830509
F1: 0.8587155963302753
ROC AUC: 0.846
